# No Power For How Long???

**Name(s)**: Andrew Zhou

**Website Link**: https://azboidynasty.github.io/power-outage-analysis/

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from lec_utils import * # Feel free to uncomment and use this. It'll make your plotly graphs look like ours in lecture!

## Step 1: Introduction

In [2]:
# TODO

outages = pd.read_csv('outage.csv')
outages

# For a location and month (time of year), how long do outages occur?

,OBS,YEAR,MONTH,U.S._STATE,...,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,1,2011,7.0,Minnesota,...,0.60,91.59,8.41,5.48
1,2,2014,5.0,Minnesota,...,0.60,91.59,8.41,5.48
2,3,2010,10.0,Minnesota,...,0.60,91.59,8.41,5.48
...,...,...,...,...,...,...,...,...,...
1531,1532,2009,8.0,South Dakota,...,0.15,98.31,1.69,1.69
1532,1533,2009,8.0,South Dakota,...,0.15,98.31,1.69,1.69
1533,1534,2000,NaN,Alaska,...,0.02,85.76,14.24,2.90


## Step 2: Data Cleaning and Exploratory Data Analysis

In [3]:
columns_to_keep = ['MONTH', 'POSTAL.CODE', 'OUTAGE.DURATION']
outages = outages[columns_to_keep]
outages

,MONTH,POSTAL.CODE,OUTAGE.DURATION
0,7.0,MN,3060.0
1,5.0,MN,1.0
2,10.0,MN,3000.0
...,...,...,...
1531,8.0,SD,59.0
1532,8.0,SD,181.0
1533,NaN,AK,NaN


A power outage that lasts 0 minutes shouldn't really count as an outage. Also, an outage that lasts 1 minute isn't something most people would be too concerned about. Let's drop any outages that last 0 or 1 minutes.

In [4]:
# TODO

# A power outage that last 0 minutes doesn't count as a power outage

outages = outages[outages['OUTAGE.DURATION'] != 0]

# Also drop 1 minute outages
outages = outages[outages['OUTAGE.DURATION'] != 1]
outages

,MONTH,POSTAL.CODE,OUTAGE.DURATION
0,7.0,MN,3060.0
2,10.0,MN,3000.0
3,6.0,MN,2550.0
...,...,...,...
1531,8.0,SD,59.0
1532,8.0,SD,181.0
1533,NaN,AK,NaN


In [5]:
outages['MONTH'].isna().sum() , outages['POSTAL.CODE'].isna().sum()

(9, 0)

We see that there are no missing values for state postal code.

Let's try grouping by state to further assist with data analysis and cleaning, especially dealing with missing values.

In [6]:
grouped = outages.groupby('POSTAL.CODE')
agg_result = grouped.agg(
    missing_months=('MONTH', lambda x: x.isna().sum()),
    mean_outage_duration=('OUTAGE.DURATION', 'mean'),
    count_power_outage=('POSTAL.CODE', 'count'),
    missing_outage_duration=('OUTAGE.DURATION', lambda x: x.isna().sum())
).reset_index()
agg_result

,POSTAL.CODE,missing_months,mean_outage_duration,count_power_outage,missing_outage_duration
0,AK,1,NaN,1,1
1,AL,1,1152.80,6,1
2,AR,0,1577.42,24,0
...,...,...,...,...,...
47,WI,0,7904.11,20,1
48,WV,0,9305.00,3,0
49,WY,0,66.33,3,0


One of our independent variables (month) has some missing values.

However, there are only 9 missing values for month out of over 1300 outages, and where these missing values occur doesn't seem to depend on state. Imputation is probably unnecessary here, and we can safely drop these missing values.

In [7]:
outages = outages.dropna(subset=['POSTAL.CODE'])
outages = outages.dropna(subset=['MONTH'])
outages

,MONTH,POSTAL.CODE,OUTAGE.DURATION
0,7.0,MN,3060.0
2,10.0,MN,3000.0
3,6.0,MN,2550.0
...,...,...,...
1529,12.0,ND,720.0
1531,8.0,SD,59.0
1532,8.0,SD,181.0


In [8]:
outages['OUTAGE.DURATION'].isna().sum()

49

We still have missing values for outage duration.

Note that the only outage in Alaska in this dataset has a missing value for outage duration. Imputation might not be appropriate since we don't have a reliable method to determine the best value to impute (e.g. mean outage duration).

For other states, the missing values appear to be missing at random, and a relatively small proporton of values are missing; we can drop these.

In [9]:
#clean missing outage duration values

outages = outages.dropna(subset=['OUTAGE.DURATION'])
outages

,MONTH,POSTAL.CODE,OUTAGE.DURATION
0,7.0,MN,3060.0
2,10.0,MN,3000.0
3,6.0,MN,2550.0
...,...,...,...
1529,12.0,ND,720.0
1531,8.0,SD,59.0
1532,8.0,SD,181.0


In [10]:
fig = px.histogram(
    outages,
    x="OUTAGE.DURATION",
    nbins=100,  # You can adjust the number of bins
    title="Distribution of Outage Durations"
)

fig.show()

It appears that most power outages last a shorter amount of time, with a few more signifcant ones sprinkled in. It's important to keep in mind that this distribution is asymmetrical when we work with this later.

In [11]:
outages.plot(kind='hist', x='MONTH', title='Distribution of when outages occur')

In [12]:
outages['POSTAL.CODE'].value_counts().plot(kind='bar', title='Distribution of where outages occur')

In [13]:
fig = px.box(
    outages,
    x='MONTH',
    y='OUTAGE.DURATION',
    title="Outage Duration vs Month"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Outage Duration",
)

fig.show()

Across the United States, with the exception of a few outliers here and there, the distributions of outages durations by month appear to be similar.

Outliers have a bad rap, and most of the time, our goal is to remove them. In the context of our dataset, although they are relatively uncommon, they inevitably happen (e.g. natural disaster, severe weather, other anomalies), and we must be prepared for them and take them into account.

Now, let's take a look at what happens when we examine one state in particular.

In [14]:
florida_outages = outages[outages['POSTAL.CODE'] == 'FL']
fig = px.box(
    florida_outages,
    x='MONTH',
    y='OUTAGE.DURATION',
    title="Outage Duration vs Month in Florida"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Outage Duration",
)

fig.show()

fig.write_html("assets/outage_duration_vs_month_florida.html")

Florida is infamous for being struck by significantly more hurricanes than most other states. If we analyze the distributions of outage durations by month in Florida, we can see that August, September, and October, the peak of hurricane season, have the longest power outages, where storms can knock out power for several days at a time.

The United States is such a large and geographically diverse country, which makes it basically impossible to generalize outage durations for the entire country. This specific plot tells us that location matters when predicting outage duration.

## Step 3: Framing a Prediction Problem

For a month of the year and a location inside the country, how long should I expect a power outage there and then to last?

This is a regression problem, as we are predicting outage duration, a continuously distributed quantitative variable.

Outage duration is a natural response variable, and predicting it is helpful in many ways. Most importantly, it could help energy providers improve resource allocation to restore power as quickly as possible depending on time and location.

Mean absolute error is likely the best metric for model performance evaluation. MAE returns the average error in absolute terms, in the same units as outage duration. Additionally, since we elected not to drop outliers, MAE is also resistant to outliers, meaning that outliers won't disproportionately affect our evaluation metrics.

Mean squared error, unfortunately, is sensitive to outliers. $R^2$, while great for linear models, can be misleading for non-linear models, and is not as straightforward to interpret as MAE.

## Step 4: Baseline Model

We'll fit a baseline model, a basic least squares regression model.

We have an ordinal feature, month, conveniently already represented numerically for us.

We also have a nominal feature, state postal code, which we can one hot encode.

Our response variable, outage duration, is a quantitative variable, as mentioned earlier.

In [15]:
# TODO

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

encoder = OneHotEncoder(sparse_output=False, drop='first')  # Avoid multicollinearity
states_encoded = encoder.fit_transform(outages[["POSTAL.CODE"]])

# Combine encoded states with other features (MONTH)
X = pd.DataFrame(states_encoded, columns=encoder.get_feature_names_out(["POSTAL.CODE"]))
X["MONTH"] = outages["MONTH"].values

# X = outages[['MONTH', 'POSTAL.CODE']]
y = outages['OUTAGE.DURATION']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
model_baseline = LinearRegression()
model_baseline.fit(X_train, y_train)

# Make predictions
y_pred_train = model_baseline.predict(X_train)
y_pred_test = model_baseline.predict(X_test)

# Evaluate the model
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

train_mae, test_mae

(2843.2781578934982, 2994.7488956539687)

In [17]:
from sklearn.metrics import r2_score
r2_baseline = r2_score(y_test, y_pred_test)
r2_baseline

-0.034433313445643376

In [18]:
mae_baseline_relative = test_mae / y_test.mean()
mae_baseline_relative

1.0662645067971608

Our MAE turns out to be nearly 3000 minutes, about 50 hours. Our relative MAE, the ratio of the MAE to the mean of the response variable, is over 1. Additionally, $R^2 < 1$. Our baseline model performs quite poorly.

In [19]:
from sklearn.model_selection import cross_val_score

baseline_pipeline = Pipeline([
    ('model', LinearRegression())   # Baseline Linear Regression model
])

# Cross-validation on the best estimator
cv_scores_base = cross_val_score(
    baseline_pipeline,
    X,                           # Full feature set
    y,                           # Full target set
    cv=5,                        # Number of folds
    scoring='neg_mean_absolute_error',  # Evaluate using MAE
    n_jobs=-1                    # Parallel processing
)

# Report results
cv_mae_base = -cv_scores_base.mean()  # Convert from negative MAE
cv_std_base = cv_scores_base.std()    # Standard deviation of MAE

print(f"Cross-validated MAE: {cv_mae_base:.2f} ± {cv_std_base:.2f}")


Cross-validated MAE: 5044377180195318.00 ± 10088754360385132.00


In addition to a model that performs poorly on this data, we have a very large cross-validated MAE, indicating that furthermore, our baseline model does not generalize well to unseen data, presumably because it also fails to perform well on existing data.

## Step 5: Final Model

Let's try to improve our model, this time using RandomForestRegressor, which is good for handling different types of features as well as different types of relationships, including non-linear, as well as being resistant to outliers.

Remember that the distribution of outage durations is skewed right. We'd ideally want a symmetric distribution, so let's apply a log transformation.

Note that month is represented as whole numbers from 1 to 12. Although random forests handle non-scaled features quite well, and it likely won't make a siginificant difference, transforming this to a normal distribution shouldn't hurt.

It's also likely that outage durations are more influenced by season rather than month. For instance, outages in January and February might be similar in duration due to similarities in cause like weather factors. We can create a new feature for seasons based on month.

In [20]:
# TODO

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, QuantileTransformer, PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from numpy import log1p, expm1

# Helper function to map months to seasons
def month_to_season(month):
    if month in [12, 1, 2]:  # Winter
        return 'Winter'
    elif month in [3, 4, 5]:  # Spring
        return 'Spring'
    elif month in [6, 7, 8]:  # Summer
        return 'Summer'
    else:  # Fall
        return 'Fall'

# Apply month-to-season transformation
outages['SEASON'] = outages['MONTH'].apply(month_to_season)

X['SEASON'] = outages['SEASON'].values
y = outages['OUTAGE.DURATION']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#save a copy of original
y_train_0 = y_train
y_test_0 = y_test
X_test_0 = X_test

# log transform since outage duration is skewed right
y_train = log1p(y_train)
y_test = log1p(y_test)

# Define categorical and numerical columns
# categorical_cols = ['POSTAL.CODE', 'SEASON']
categorical_cols = [col for col in X_train.columns if col.startswith('POSTAL.CODE')]
numerical_cols = ['MONTH']

# Preprocessing steps for each type of column
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')
numerical_transformer = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler', StandardScaler()),
    ('quantile', QuantileTransformer(output_distribution='normal'))
])

# ColumnTransformer to preprocess features
# preprocessor = ColumnTransformer([
#     ('cat', categorical_transformer, categorical_cols),
#     ('num', numerical_transformer, numerical_cols)
# ])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ]
)

# Define the model
model = RandomForestRegressor(random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Define hyperparameters for tuning
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [10, 20, None],
    'model__min_samples_split': [2, 5, 10]
}

# Perform GridSearchCV
grid_search = GridSearchCV(
    pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1
)

grid_search.fit(X_train, y_train)

# Best parameters and performance
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Convert from negative MAE
best_params, best_score


Fitting 5 folds for each of 27 candidates, totalling 135 fits


/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (832). n_quantiles is set to n_samples.
  warnings.warn(
/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (832). n_quantiles is set to n_samples.
  warnings.warn(
/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (832). n_quantiles is set to n_samples.
  warnings.warn(
/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (832). n_quantiles is set to n_samples.
  warnings.warn(
/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10

({'model__max_depth': 20,
  'model__min_samples_split': 10,
  'model__n_estimators': 200},
 1.3705290484700026)

In [21]:
y_pred_test_0 = grid_search.best_estimator_.predict(X_test_0)
y_pred_test = grid_search.best_estimator_.predict(X_test)

test_mae_0 = mean_absolute_error(y_test_0, y_pred_test_0)
test_mae = mean_absolute_error(y_test, y_pred_test)
relative_mae = test_mae / y_test.mean()
relative_mae_0 = test_mae_0 / y_test_0.mean()

print(f"Mean Absolute Error (MAE): {test_mae_0:.2f}")
print(f"Relative MAE: {relative_mae:.2%}")
print(f"Relative MAE in original space: {relative_mae_0:.2%}")

Mean Absolute Error (MAE): 2802.03
Relative MAE: 20.62%
Relative MAE in original space: 99.76%


In [22]:
cv_scores = cross_val_score(
    grid_search.best_estimator_,  # Best model from GridSearchCV
    X,                           # Full feature set
    y,                           # Full target set
    cv=5,                        # Number of folds
    scoring='neg_mean_absolute_error',  # Evaluate using MAE
    n_jobs=-1                    # Parallel processing
)

# Report results
cv_mae = -cv_scores.mean()  # Convert from negative MAE
cv_std = cv_scores.std()    # Standard deviation of MAE

print(f"Cross-validated MAE: {cv_mae:.2f} ± {cv_std:.2f}")


Cross-validated MAE: 2728.48 ± 679.21


/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [20, 21, 39, 45, 46] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [5, 6, 10, 15, 22, 24, 25, 29, 33, 35, 37, 41] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [1, 12, 13, 18, 23, 44] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/andrewzhou/miniforge3/envs/pds/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 3, 4, 8, 9, 1

Using GridSearchCV to tune hyperparameters in the ranges above, the best hyperparemeters turned out to be:

'model__max_depth': 20
  
  'model__min_samples_split': 10
  
  'model__n_estimators': 200

Our new MAE, although still not ideal, is a slight improvement, decreasing to around 2800 minutes, or around 46.67 hours.

In the original feature space, we have a slightly improved, but not excellent, relative MAE, being able to just about decrease it to under 100%.

On the other hand, in the transformed space, our relative MAE is at around 20%, which indicates very good performance in the transformed space. This is expected, since after feature engineering, we can more easily work with standardized and symmetric distributions.

Our new cross-validated MAE is fortunately much lower, so hopefully, our new model generalizes well to unseen data.